In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import seaborn as sns
from keras.layers import Dense, LSTM, TimeDistributed
py.init_notebook_mode(connected=True)
%matplotlib inline

Using TensorFlow backend.


In [2]:
data = pd.read_csv(filepath_or_buffer="./Data/growth_rate.csv", index_col="date")

In [3]:
data = data.apply(pd.to_numeric, errors = "coerce")

In [4]:
data['spindx'].replace(0, np.nan, inplace=True)
data['spindx'].fillna(method='ffill', inplace=True)
from sklearn.preprocessing import MinMaxScaler

In [5]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [6]:
number_of_variables = 8
values = data[['spindx'] + ['TCMNOM_Y2'] + ['TCMNOM_Y10'] + ['DCOILBRENTEU'] + ['GOLDPMGBD228NLBM'] + ['exalus'] + ['exjpus'] + ['exukus']].values
values = values.astype('float32')

In [7]:
#getting the last 3 days' data
historic_data = np.array([])
for day in values[-3:]:
    historic_data = np.concatenate((historic_data, day), axis=None)

In [8]:
#historic_data = 
historic_data.shape

(24,)

In [9]:
# look_back = 3 
# time_steps = 1
# series_to_supervised(values, look_back, time_steps)
# reframed = series_to_supervised(values, look_back, time_steps)

In [10]:
# reframed = reframed.append(dict(zip(reframed.columns, historic_data)), ignore_index=True)

In [11]:
pred_para = historic_data.reshape(1,1,historic_data.shape[0])

In [12]:
pred_para.shape

(1, 1, 24)

In [13]:
from keras.models import load_model

multi_model = load_model("8to8market_lstm_growth_rate_model_weights.hdf5")
yhat = multi_model.predict(pred_para)

In [14]:
yhat

array([[[-0.0008352 , -0.00195253,  0.00044797,  0.00177827,
          0.00067272, -0.00095498,  0.00179116,  0.00041693]]],
      dtype=float32)